In [1]:
# Importar o drive para importar o arquivo das pastas
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# BUSCA COMPLETA

In [2]:
import os
import csv
import re
import time
import spacy
nlp = spacy.load("en_core_web_sm")

FILE_KW = "/content/drive/My Drive/TCC 2025/Base_de_dados/saida_palavras_flexionadas.csv"
ARQUIVO_HISTORIAS = "/content/drive/My Drive/TCC 2025/Base_de_dados/g21-badcamp.txt"

def carregar_keywords_com_qes(caminho_arquivo):
    inicio = time.time()
    palavras_qes = {}
    if os.path.exists(caminho_arquivo):
        with open(caminho_arquivo, "r", encoding="utf-8") as arquivo_csv:
            leitor = csv.DictReader(arquivo_csv)
            for linha in leitor:
                palavra = linha['palavra'].strip()
                if palavra:
                    qes = [qe for qe, valor in linha.items() if qe.startswith("QE") and valor.strip() == '1']
                    palavras_qes[palavra] = qes
    else:
        print("Arquivo não encontrado.")
    fim = time.time()
    print(f"Tempo para carregar keywords: {fim - inicio:.4f} segundos")
    return palavras_qes

def extrair_historia_usuario(caminho_arquivo):
    inicio = time.time()
    historias_processadas = []
    with open(caminho_arquivo, "r", encoding="utf-8") as arquivo:
        for linha in arquivo:
            linha = linha.strip()
            if linha:
                if ", I want " in linha:
                    historia_usuario = linha.split(", I want ", 1)[-1]
                    historias_processadas.append(historia_usuario)
                elif "I want " in linha:
                    historia_usuario = linha.split("I want ", 1)[-1]
                    historias_processadas.append(historia_usuario)
    fim = time.time()
    print(f"Tempo para extrair histórias: {fim - inicio:.4f} segundos")
    return historias_processadas

def buscar_qes_em_historia(historia_usuario, palavras_qes):
    palavras_encontradas = []
    qes_total_BD = set()
    dic_palavra_qe_BD = {}

    # Transforma as palavras-chave em um set para busca rápida
    palavras_chave_set = set(palavras_qes.keys())

    # Tokeniza a história em palavras, mantendo apenas palavras simples
    palavras_historia = re.findall(r'\b\w+\b', historia_usuario.lower())

    for palavra in palavras_historia:
        if palavra in palavras_chave_set:
            if palavra not in dic_palavra_qe_BD:  # evita repetir
                palavras_encontradas.append(palavra)
                qes = palavras_qes.get(palavra, [])
                qes_total_BD.update(qes)
                dic_palavra_qe_BD[palavra] = qes

    return palavras_encontradas, dic_palavra_qe_BD, sorted(qes_total_BD)

def buscar_qes_em_historia_lemmatizada(historia_usuario, palavras_qes):
    # Mapear lemas das palavras-chave para as palavras originais
    palavras_chave_lem = {}
    for palavra in palavras_qes.keys():
        if palavra.strip() != "":
            doc = nlp(palavra.lower())
            if len(doc) > 0:
                palavras_chave_lem[doc[0].lemma_] = palavra

    # Tokenização crua
    palavras_cruas = re.findall(r'\b\w+\b', historia_usuario.lower())

    palavras_encontradas = []
    dic_palavra_qe = {}
    qes_total = set()

    for palavra in palavras_cruas:
        # Verificação explícita para "data"
        if palavra == "data":
            if "data" not in dic_palavra_qe and "data" in palavras_qes:
                palavras_encontradas.append("data")
                qes = palavras_qes.get("data", [])
                qes_total.update(qes)
                dic_palavra_qe["data"] = qes
            continue

        # Lematiza a palavra crua
        doc_token = nlp(palavra)
        if len(doc_token) == 0:
            continue
        token = doc_token[0]
        lema = token.lemma_

        if lema in palavras_chave_lem:
            palavra_original = palavras_chave_lem[lema]
            if palavra_original not in dic_palavra_qe:
                palavras_encontradas.append(palavra_original)
                qes = palavras_qes.get(palavra_original, [])
                qes_total.update(qes)
                dic_palavra_qe[palavra_original] = qes

    return palavras_encontradas, dic_palavra_qe, sorted(qes_total)




# Execução principal
palavras_qes = carregar_keywords_com_qes(FILE_KW)
palavras_chave = list(palavras_qes.keys())
historias_extraidas = extrair_historia_usuario(ARQUIVO_HISTORIAS)
#historias_extraidas = historias_extraidas[:1]

inicio = time.time()
for idx, historia_usuario in enumerate(historias_extraidas, start=1):
    print(f"\n{'='*60}")
    print(f"História {idx}: {historia_usuario}\n")
    print("Palavras-chave encontradas e suas QEs correspondentes:")

    palavras_encontradas, dic_palavra_qe_BD, qes_total = buscar_qes_em_historia(historia_usuario, palavras_qes)

    for palavra, qes in dic_palavra_qe_BD.items():
       print(f" - {palavra}: {qes}")

    print("\nConjunto total de QE's encontradas:")
    print(qes_total)

fim = time.time()
print(f"Tempo para busca direta: {fim - inicio:.4f} segundos")

# Arquivo de saída
ARQUIVO_SAIDA = "/content/drive/My Drive/TCC 2025/Base_de_dados/qes_por_historia_busca_direta_G21.txt"
quant_busca_direta = []

inicio = time.time()
with open(ARQUIVO_SAIDA, "w", encoding="utf-8") as arquivo_saida:
    for idx, historia_usuario in enumerate(historias_extraidas, start=1):
        palavras_encontradas, dic_palavra_qe_BD, qes_total = buscar_qes_em_historia(historia_usuario, palavras_qes)

        # Converte o conjunto de QEs em string separada por vírgula
        if len(qes_total) != 0:
          linha_qes = ",".join(qes_total)
          quant_busca_direta.append(len(qes_total))
        else:
          linha_qes = " " #"QE1,QE10,QE11,QE12,QE13,QE14,QE15,QE16,QE17,QE18,QE2,QE3,QE4,QE5,QE6,QE7,QE8,QE9"
          quant_busca_direta.append(0)

        # Escreve no arquivo (uma linha por história)
        arquivo_saida.write(f"{linha_qes}\n")

fim = time.time()

economia_busca_direta = []
print(f"Tempo de processamento e gravação (busca direta): {fim - inicio:.4f} segundos")
#print(f"Resultados salvos em: {ARQUIVO_SAIDA}")
print(quant_busca_direta)
print(sum(quant_busca_direta))
soma = 18*len(historias_extraidas)-sum(quant_busca_direta)
print(f"Taxa de diminuição (busca direta): {1 - sum(quant_busca_direta)/(18*len(historias_extraidas))}")

# Arquivo de saída para lematização
ARQUIVO_SAIDA_LEMA = "/content/drive/My Drive/TCC 2025/Base_de_dados/qes_por_historia_lemmatizada_G21.txt"
quant_busca_lema = []

inicio = time.time()
with open(ARQUIVO_SAIDA_LEMA, "w", encoding="utf-8") as arquivo_saida:
    for idx, historia_usuario in enumerate(historias_extraidas, start=1):
        print(f"\n{'='*60}")
        print(f"História {idx}: {historia_usuario}\n")
        print("Palavras-chave (lematizadas) encontradas e suas QEs correspondentes:")

        palavras_encontradas, dic_palavra_qe_lema, qes_total_lema = buscar_qes_em_historia_lemmatizada(historia_usuario, palavras_qes)

        for palavra, qes in dic_palavra_qe_lema.items():
            print(f" - {palavra}: {qes}")

        print("\nConjunto total de QE's encontradas:")
        print(qes_total_lema)

        # Converte o conjunto de QEs em string separada por vírgula
        if len(qes_total_lema) != 0:
            linha_qes = ",".join(qes_total_lema)
            quant_busca_lema.append(len(qes_total_lema))
        else:
            linha_qes = " " #"QE1,QE10,QE11,QE12,QE13,QE14,QE15,QE16,QE17,QE18,QE2,QE3,QE4,QE5,QE6,QE7,QE8,QE9"
            quant_busca_lema.append(0)

        # Escreve no arquivo (uma linha por história)
        arquivo_saida.write(f"{linha_qes}\n")

fim = time.time()


Tempo para carregar keywords: 1.7100 segundos
Tempo para extrair histórias: 0.5560 segundos

História 1: to view a list of sponsors, so that I can thank all the awesome sponsors.

Palavras-chave encontradas e suas QEs correspondentes:
 - view: ['QE7', 'QE8', 'QE9', 'QE11']
 - list: ['QE10', 'QE11']
 - sponsors: []

Conjunto total de QE's encontradas:
['QE10', 'QE11', 'QE7', 'QE8', 'QE9']

História 2: to view a list of user profiles, so that I know who is attending the conference.

Palavras-chave encontradas e suas QEs correspondentes:
 - view: ['QE7', 'QE8', 'QE9', 'QE11']
 - list: ['QE10', 'QE11']
 - user: ['QE1', 'QE2', 'QE7', 'QE8']
 - profiles: ['QE1', 'QE2', 'QE5', 'QE7', 'QE8', 'QE9', 'QE10', 'QE11', 'QE16']
 - know: []
 - attending: []

Conjunto total de QE's encontradas:
['QE1', 'QE10', 'QE11', 'QE16', 'QE2', 'QE5', 'QE7', 'QE8', 'QE9']

História 3: to email all the trainers at once from the website for info and updates, so that It is easier to use.

Palavras-chave encontradas 